# Roll No: 2018102024

# Excercise 1

In the tutorial you saw how to compute LDA for a two class problem. In this excercise we will work on a multi-class problem. We will be working with the famous Iris dataset that has been deposited on the UCI machine learning repository
(https://archive.ics.uci.edu/ml/datasets/Iris).

The iris dataset contains measurements for 150 iris flowers from three different species.

The three classes in the Iris dataset:
1. Iris-setosa (n=50)
2. Iris-versicolor (n=50)
3. Iris-virginica (n=50)

The four features of the Iris dataset:
1. sepal length in cm
2. sepal width in cm
3. petal length in cm
4. petal width in cm

<img src="iris_petal_sepal.png">



In [1]:
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set();
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy import pi

### Importing the dataset

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']
dataset = pd.read_csv(url, names=names)

dataset.tail()

,sepal-length,sepal-width,petal-length,petal-width,Class
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


### Data preprocessing

Once dataset is loaded into a pandas data frame object, the first step is to divide dataset into features and corresponding labels and then divide the resultant dataset into training and test sets. The following code divides data into labels and feature set:

In [3]:
X = dataset.iloc[:, 0:4].values
y = dataset.iloc[:, 4].values

The above script assigns the first four columns of the dataset i.e. the feature set to X variable while the values in the fifth column (labels) are assigned to the y variable.

The following code divides data into training and test sets:

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#### Feature Scaling

We will now perform feature scaling as part of data preprocessing too. For this task, we will be using scikit learn `StandardScalar`.

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
print(X_train.shape)
print(y_train.shape)

(120, 4)
(120,)


## Write your code below

Write you code below to LDA on the IRIS dataset and compute the overall accuracy of the classifier.

In [7]:
### WRITE YOUR CODE HERE ####

def getlabel(y, labels=None):
    if labels == None: labels = [i for i in range(y.shape[0])]
    classes = np.unique(y)
    out = np.zeros((y.shape[0]), dtype='object')
    cnt = 0
    for c in classes:
        out[y == c] = labels[cnt]
        cnt += 1
    return out

def cov(X):
    return 1 / X.shape[0] * X.T @ X

def LDA(X_train, y_train):
    
    m, d = X_train.shape
    mean = np.mean(X_train, axis=0).reshape((1, -1))
    SW, SB = np.zeros((d, d)), np.zeros((d, d))
    classes = np.unique(y_train)
    for c in range(classes.shape[0]):
        Xc = X_train[y_train == classes[c]]
        meanc = np.mean(Xc, axis=0).reshape((1, -1))
        SB += Xc.shape[0] * cov(meanc - mean)
        SW += cov(Xc - meanc)
    SB /= m
    
    eigvals, eigvecs = np.linalg.eig(np.linalg.pinv(SW) @ SB)
    
    idx = np.argsort(eigvals)[::-1]
    eigvals = np.real(eigvals[idx])
    eigvecs = np.real(eigvecs[:, idx])
    
    return eigvecs[:, 0:1].T

SyntaxError: invalid syntax (<ipython-input-7-5996b967cf84>, line 14)

In [ ]:
W_lda = LDA(X_train, y_train)

In [ ]:
plt.scatter((W_lda @ X_train.T).ravel(), np.zeros((X_train.shape[0])), c=getlabel(y_train,['r', 'g', 'b']))
plt.show()

In [ ]:
def info_per_class(X, y):
    m, _ = X.shape
    X = X.ravel()
    classes = np.unique(y)
    prob, mean, std = [], [], []
    for c in classes:
        Xc = X[y == c].ravel()
        prob.append(Xc.shape[0] / m)
        mean.append(np.mean(Xc))
        std.append(np.std(Xc))
    prob = np.array(prob)
    mean = np.array(mean)
    std = np.array(std)
    return prob, mean, std

In [ ]:
prob, mean, std = info_per_class(X_train @ W_lda.T, y_train)

In [ ]:
def calc_prob(X, p, m, s):
    return 1 / s * np.exp(-1 / 2 * (((X - m) / s) ** 2))

def predict(X, prob, mean, std):
    val = []
    for c in range(mean.shape[0]):
        val.append(calc_prob(X, prob[c], mean[c], std[c]))
    val = np.array(val)
    return np.argmax(val, axis=0)

def accuracy(X, y, W_lda, prob, mean, std):
    y_pred = predict((X @ W_lda.T).ravel(), prob, mean, std)
    return np.count_nonzero(getlabel(y) == y_pred) / y.shape[0]

In [ ]:
print(f"Training accuracy: {accuracy(X_train, y_train, W_lda, prob, mean, std)}")
print(f"Test accuracy: {accuracy(X_test, y_test, W_lda, prob, mean, std)}")

In [ ]:
def plot1D_decision_boundary(X, y, prob, mean, std):
    X = X.ravel()
    label = np.unique(y)
    fig, ax = plt.subplots()
    ax.scatter(X, np.zeros((X.shape[0])), c=getlabel(y, ['r','g','b']), label=label, marker='x')
    X = np.linspace(-3, 3, 100)
    y = predict(X, prob, mean, std)
    ax.scatter(X, np.zeros((X.shape[0])), alpha=0.3, c=getlabel(y,['r','g','b']), label=label)
    ax.legend()
    plt.show()

In [ ]:
plot1D_decision_boundary(X_train @ W_lda.T, y_train, prob, mean, std)